In [1]:
# TO connect with anvil server and import media and tables from anvil 
import anvil.server
anvil.server.connect("FPNIE3WVSY7O3T4PKLT2ZEKY-A5JAS2VD4SKY72XZ")
import anvil.media
from anvil.tables import app_tables

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [2]:
# importing all necessary libraries 
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
# defing 

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils #Draw utilities 

In [4]:
from keras.models import load_model
model = load_model('cricket_pose_detection.h5')

In [5]:
# function for detecting pose of the person using MideaPipe

def multiple_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # Color convertion 
    image.flags.writeable = False
    results = model.process(image)                # Precessing the image via holistic model to generate pose keypoints
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # Color convertion again
    return image,results                          # returning generated pose keypoints as results and image 

In [6]:

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [8]:


def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

In [15]:
colors = [(245,117,16), (117,245,16), (16,117,245),(200,103,27),(200,103,27),(200,103,27)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 25ms/step


In [10]:
# defining an array that has all cricket classes
actions = np.array(['Cover Drive', 'Helicopter', 'Late Cut', 'Pull', 'Straight Drive', 'Sweep']) 

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
# for image pose detection 

@anvil.server.callable
def cricket_pose_image(file):
    sentence = []
    predictions = []
    threshold = 0.5
    with anvil.media.TempFile(file) as filename:
        cap = cv2.imread(filename)
        cap = cv2.resize(cap, (960, 540))
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        sequence = [] # creating empty list
        image, results = multiple_detection(cap, holistic)#generating pose keypoints via processing image in the holistic model

        # Draw landmarks
        draw_styled_landmarks(image, results) # drawing landmarks of the human joints and pose on the image that were generated
                                              # in the result via processing image through mediapipe

        #  Prediction logic
        keypoints = extract_keypoints(results) # extracting and storing human pose kepoints in the array 
        sequence.append(keypoints)             # appending extracted kepoints in the list


        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        predictions.append(np.argmax(res))


        #3. Viz logic
        if np.unique(predictions[-10:])[0]==np.argmax(res): 
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

       
        # Viz probabilities
        image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('Image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    

In [13]:
@anvil.server.callable
def cricket_pose_video(file):
    predictions = []
    threshold = 0.5
    with anvil.media.TempFile(file) as filename:
        cap = cv2.VideoCapture(filename)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        ret,frame=cap.read()
        while ret and cap.isOpened():
            sequence = []
            frame=cv2.resize(frame, (960, 540))
            image, results = multiple_detection(frame, holistic)
           
            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)

            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))


            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

            # Show to screen
            cv2.imshow('Video', image)

            ret,frame=cap.read()

            # Break gracefully
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

    

In [14]:
@anvil.server.callable
def cricket_pose_live():
    predictions = []
    threshold = 0.5
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.6,min_tracking_confidence=0.6) as holistic:
        ret,frame=cap.read()
        while ret and cap.isOpened():
            sequence = []
            frame=cv2.resize(frame, (960, 540))
            image, results = multiple_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)

            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))


            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

            # Show to screen
            cv2.imshow('Live', image)

            ret,frame=cap.read()

            # Break gracefully
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
        